In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import os
from src.progress_bar import printProgressBar

for dirname, _, filenames in os.walk('../../data/lyrics'):
    for filename in filenames:
        print(os.path.join(dirname, filename).replace("\\","/"))

print(50*'-')

for dirname, _, filenames in os.walk('../../data/billboard'):
    for filename in filenames:
        print(os.path.join(dirname, filename).replace("\\","/"))




../../data/lyrics/artist_song_lyrics.csv
../../data/lyrics/bb-t100-lyrics.csv
../../data/lyrics/bb-t100-lyrics_new.csv
../../data/lyrics/bb_t100_lyrics_en.csv
../../data/lyrics/bb_t100_lyrics_en_v2.csv
../../data/lyrics/bb_t100_lyrics_en_v3.csv
../../data/lyrics/corona-lyrics.csv
../../data/lyrics/lyrics_invalid.json
../../data/lyrics/lyrics_invalid_updated.csv
../../data/lyrics/backups/artist_song_lyrics_bak.csv
--------------------------------------------------
../../data/billboard/bb_t100_en.csv
../../data/billboard/billboard.csv
../../data/billboard/billboard_top_100_2020.csv
../../data/billboard/backups/bb-t100_backup.csv


In [26]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 30)
lyrics = pd.read_csv('../../data/lyrics/bb-t100-lyrics.csv', index_col=0)
lyrics.head()

,billboard_id,lyrics_id,artist,first_artist,song,weeks_on_chart,peak_rank,lyrics,url,length,word_count,language,language_score,first_appearance
0,0,0,Ariana Grande,Ariana Grande,"Thank U, Next",28.0,1.0,Thought I'd end up with Se...,https://genius.com/Ariana-...,2409,460,en,0.999997,2019-01-05
1,1,1,Halsey,Halsey,Without Me,52.0,1.0,Found you when your heart ...,https://genius.com/Halsey-...,2095,435,en,0.999995,2019-01-05
2,2,2,Mariah Carey,Mariah Carey,All I Want For Christmas I...,43.0,1.0,I don't want a lot for Chr...,https://genius.com/Mariah-...,1918,388,en,0.999996,2019-01-05
3,3,3,Travis Scott,Travis Scott,Sicko Mode,52.0,1.0,"Astro, yeah\nSun is down, ...",https://genius.com/Travis-...,3943,771,en,0.999998,2019-01-05
4,4,4,Post Malone & Swae Lee,Post Malone,Sunflower (Spider-Man: Int...,53.0,1.0,"Ayy, ayy, ayy, ayy (Ooh)\n...",https://genius.com/Post-ma...,1534,305,en,0.999997,2019-01-05


In [27]:
bb_t100 = pd.read_csv('../../data/billboard/billboard.csv', index_col=0)
bb_t100.rename(columns={'id' : 'billboard_id'}, inplace=True)
bb_t100.head()

,billboard_id,song,artist,peak_rank,rank_last_week,weeks_on_chart,rank,date
0,0,"Thank U, Next",Ariana Grande,1,1,8,1,2019-01-05
1,1,Without Me,Halsey,2,2,12,2,2019-01-05
2,2,All I Want For Christmas I...,Mariah Carey,3,7,30,3,2019-01-05
3,3,Sicko Mode,Travis Scott,1,3,21,4,2019-01-05
4,4,Sunflower (Spider-Man: Int...,Post Malone & Swae Lee,4,4,10,5,2019-01-05


In [28]:
merge_cols = lyrics.columns.difference(bb_t100.columns)
merge_cols = merge_cols.tolist()
merge_cols.append('billboard_id')
merge_cols

['first_appearance',
 'first_artist',
 'language',
 'language_score',
 'length',
 'lyrics',
 'lyrics_id',
 'url',
 'word_count',
 'billboard_id']

In [29]:
merge_cols = ['language','language_score','lyrics_id','billboard_id', 'first_appearance',]
bb_t100 = bb_t100.merge(lyrics[merge_cols],how='left', left_on='billboard_id', right_on='billboard_id', copy=False)
bb_t100 = bb_t100.sort_values(by=['artist', 'song', 'lyrics_id']).interpolate(method='pad')
bb_t100['lyrics_id'] = pd.to_numeric(bb_t100['lyrics_id'], downcast='integer')
bb_t100['date'] = pd.to_datetime(bb_t100['date'], format='%Y-%m-%d')
bb_t100.sort_values(by='billboard_id', inplace=True)
bb_t100 = bb_t100[['billboard_id',
                   'lyrics_id',
                   'song',
                   'artist',
                   'peak_rank',
                   'rank_last_week',
                   'weeks_on_chart',
                   'rank',
                   'date',
                   'first_appearance',
                   'language',
                   'language_score']]
bb_t100.head()

,billboard_id,lyrics_id,song,artist,peak_rank,rank_last_week,weeks_on_chart,rank,date,first_appearance,language,language_score
0,0,0,"Thank U, Next",Ariana Grande,1,1,8,1,2019-01-05,2019-01-05,en,0.999997
1,1,1,Without Me,Halsey,2,2,12,2,2019-01-05,2019-01-05,en,0.999995
2,2,2,All I Want For Christmas I...,Mariah Carey,3,7,30,3,2019-01-05,2019-01-05,en,0.999996
3,3,3,Sicko Mode,Travis Scott,1,3,21,4,2019-01-05,2019-01-05,en,0.999998
4,4,4,Sunflower (Spider-Man: Int...,Post Malone & Swae Lee,4,4,10,5,2019-01-05,2019-01-05,en,0.999997


In [30]:
bb_t100_en = bb_t100.loc[bb_t100['language'] == 'en']
# bb_t100_en.reset_index(drop=True, inplace=True)
bb_t100_en.to_csv('../../data/billboard/bb_t100_en.csv', encoding='utf-8')

lyrics_en = lyrics.loc[lyrics['language'] == 'en']
# lyrics_en.reset_index(drop=True, inplace=True)
lyrics_en.to_csv('../../data/lyrics/bb_t100_lyrics_en.csv', encoding='utf-8')

In [46]:
bb_t100_new = pd.read_csv('../../data/lyrics/bb-t100-lyrics_new.csv', index_col=0)
bb_t100_new.head()
bb_t100_new_lat = bb_t100_new[['lyrics', 'genius_description', 'genius_annotations', 'genius_comments','release_date']]
print(bb_t100_new_lat.loc[1183:1183].to_latex())

\begin{tabular}{llllll}
\toprule
{} &                         lyrics &             genius\_description &             genius\_annotations &                genius\_comments & release\_date \\
\midrule
1183 &  Yeah\textbackslash n\textbackslash nI've been tryna ca... &  “Blinding Lights” is the s... &  [('You can turn me on with... &  ['This gave so much ”False... &   2019-11-29 \\
\bottomrule
\end{tabular}



In [37]:
bb_t100_new.loc[bb_t100_new['song'] == 'Blinding Lights']

,id,artist,first_artist,song,weeks_on_chart,peak_rank,lyrics,url,genius_id,genius_primary_artist,genius_description,genius_annotations,genius_comments,release_date
1183,4910,The Weeknd,The Weeknd,Blinding Lights,55,1,Yeah\n\nI've been tryna ca...,https://genius.com/The-wee...,5049949.0,"The Weeknd, 0 songs",“Blinding Lights” is the s...,[('You can turn me on with...,['This gave so much ”False...,2019-11-29
